In [75]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import sklearn as sk
from sklearn import decomposition as dec
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.utils.prune as prune

In [80]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [81]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=12, kernel_size=(5, 5))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=12, out_channels=16, kernel_size=(5, 5))
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [82]:
path = './cifar_net.pt'
model = NN()
model.load_state_dict(torch.load(path))

model.eval()

NN(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [83]:
#pruning 

prune.random_unstructured(model.fc1, name='weight', amount=0.5)
print(model.fc1._forward_pre_hooks)


OrderedDict([(8, <torch.nn.utils.prune.RandomUnstructured object at 0x0000015D000679A0>)])


In [84]:
datasample = iter(testloader)
img, lbl = datasample.next()
output = model(img)

print(output)
acc, pred = torch.max(output, dim=1)
print(pred)

tensor([[ 0.4647, -0.8140, -0.5935,  2.1472, -0.4027,  1.0611, -0.2714, -0.9863,
          0.3386, -0.6738],
        [ 2.9563,  3.4883,  0.2924, -1.5808, -1.5086, -2.9230, -2.0223, -4.0567,
          3.8015,  0.1930],
        [ 4.2275, -0.2214,  1.1828,  0.4962, -0.2673, -1.1644, -3.1107, -2.9174,
          1.4882, -0.2510],
        [ 5.0329,  1.6930,  0.1772, -0.7826, -0.4201, -2.7751, -2.9997, -3.6510,
          2.9027,  0.7673]], grad_fn=<AddmmBackward0>)
tensor([3, 8, 0, 0])


In [80]:
fc_1 = model.fc1.weight.detach().numpy().T


new_nmf = dec.NMF(n_components=60, init='random', random_state=42)
fc_1_reduced = new_nmf.fit_transform(fc_1)


done!!


In [87]:
model.fc1.weight = nn.Parameter(torch.tensor(fc_1.T))
print(model.fc1.wei.shape)

torch.Size([120, 400])


In [96]:
from torch.profiler import profile, record_function, ProfilerActivity
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

print(optimizer)

running_loss = 0.0
total = 0.0
correct = 0.0
loss_nodr =[]
acc_nodr = []
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

for epoch in range(5):
    
    print('epoch:', epoch)
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad(True)
        with torch.set_grad_enabled(True):
            with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
             outputs = model(inputs).to(device)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        running_loss += loss.item()
        if i % 2000 == 1999:
            loss_temp = running_loss / 2000
            acc_temp = 100 * correct / total
            loss_nodr.append(loss_temp)
            acc_nodr.append(acc_temp)
            print(
                f'[{epoch + 1}, {i + 1:5d}] loss: {loss_temp:.3f} acc: {acc_temp:.3f}')
            running_loss = 0.0
            correct = 0.0
            total = 0.0
print('Finished Training')


SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0


c:\Users\sulta\anaconda3\lib\site-packages\torch\autograd\profiler.py:160: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


[1,  2000] loss: 0.706 acc: 74.537
[1,  4000] loss: 0.719 acc: 74.525
[1,  6000] loss: 0.730 acc: 73.425
[1,  8000] loss: 0.763 acc: 72.787
[1, 10000] loss: 0.770 acc: 72.775
[1, 12000] loss: 0.778 acc: 72.812
epoch: 1
[2,  2000] loss: 0.849 acc: 75.910
[2,  4000] loss: 0.703 acc: 75.362
[2,  6000] loss: 0.716 acc: 74.800
[2,  8000] loss: 0.756 acc: 73.513
[2, 10000] loss: 0.746 acc: 73.737
[2, 12000] loss: 0.758 acc: 73.100
epoch: 2
[3,  2000] loss: 0.839 acc: 75.900
[3,  4000] loss: 0.689 acc: 75.325
[3,  6000] loss: 0.702 acc: 75.188
[3,  8000] loss: 0.728 acc: 74.025
[3, 10000] loss: 0.731 acc: 74.400
[3, 12000] loss: 0.746 acc: 73.800
epoch: 3
[4,  2000] loss: 0.831 acc: 76.480
[4,  4000] loss: 0.673 acc: 75.938
[4,  6000] loss: 0.687 acc: 75.412
[4,  8000] loss: 0.716 acc: 74.300
[4, 10000] loss: 0.705 acc: 75.312
[4, 12000] loss: 0.728 acc: 74.088
epoch: 4
[5,  2000] loss: 0.809 acc: 77.430
[5,  4000] loss: 0.665 acc: 76.312
[5,  6000] loss: 0.666 acc: 76.550
[5,  8000] loss: 0.

In [97]:
print(model.fc1.weight)

tensor([[ 0.3503,  0.0000,  0.0000,  ...,  0.0343,  0.1725, -0.2020],
        [ 0.3041, -0.0000, -0.1218,  ...,  0.1513,  0.0000, -0.0000],
        [-0.0000, -0.0000, -0.2168,  ..., -0.2211, -0.0000, -0.0336],
        ...,
        [ 0.0786, -0.2009,  0.0000,  ...,  0.0000, -0.1257,  0.2552],
        [ 0.0000, -0.0000, -0.0000,  ...,  0.0329, -0.0000,  0.0000],
        [-0.4160, -0.5303, -0.0000,  ...,  0.0000,  0.1677,  0.1846]],
       grad_fn=<MulBackward0>)


In [92]:
fc_1 = model.fc1.weight.detach().numpy().T

for a in range(fc_1.shape[0]):
    for b in range(fc_1.shape[1]):
        if(fc_1[a][b]<=0): fc_1[a][b] = abs(0.0)
        



# new_nmf = dec.NMF(n_components=60, init='random', random_state=42)
# fc_1_reduced = new_nmf.fit_transform(fc_1)


[0.34771964 0.28580782 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.09259516 0.         0.         0.
 0.04895765 0.21041448 0.31263116 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.1393346  0.         0.         0.2647249  0.         0.
 0.         0.         0.         0.         0.         0.
 0.28365788 0.         0.07291234 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.09790394 0.         0.         0.         0.         0.
 0.         0.         0.01413102 0.         0.         0.
 0.         0.04597853 0.         0.         0.07514965 0.
 0.04118858 0.         0.06643911 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.04860302 0.         0.09110495 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.05913865 0.24707364 0.         0.         